<a href="https://colab.research.google.com/github/srush/Tensor-Puzzles-Penzai/blob/main/Tensor_Puzzlers_Penzai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor Puzzles - Penzai Edition
- by [Sasha Rush](http://rush-nlp.com) - [srush_nlp](https://twitter.com/srush_nlp)


This is a version of the [tensor puzzles](https://github.com/srush/Tensor-Puzzles) implemented the JAX [Penzai](https://penzai.readthedocs.io/en/stable/notebooks/named_axes.html) library. Available on [Github](https://github.com/srush/Tensor-Puzzles-Penzai).

Penzai is a really nice fit for these puzzles both because it comes with a really clean visualization library built-in and because it has a very nice named-tensor implementation.


I recommend running in [Colab](https://colab.research.google.com/github/srush/Tensor-Puzzles-Penzai/blob/main/Tensor_Puzzlers_Penzai.ipynb).


In [3]:
!pip install -qqq jaxtyping hypothesis pytest penzai
import jax.numpy as np
import numpy as onp
from penzai import pz
arange = pz.nx.arange
where = pz.nx.nmap(np.where)
wrap = pz.nx.wrap
pz.ts.register_as_default()

# Optional automatic array visualization extras:
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()
pz.ts.active_autovisualizer.set_interactive(pz.ts.ArrayAutovisualizer(force_continuous=True, around_zero=True,  prefers_column=["j"], prefers_row=["i"]))

In [4]:
import inspect
import random
from jaxtyping import Int32
NamedArray = pz.nx.NamedArray
def make_test(name, problem, problem_spec, add_sizes=[],
              init_size = {},
              constraint=lambda d: d):
    args = {}
    signature = inspect.signature(problem)
    for n, p in signature.parameters.items():
        args[n] = [d.name for d in p.annotation.dims]
    args["return"] = [d.name for d in signature.return_annotation.dims]

    def make_instance():
        example = {}
        reg = {}
        sizes = {}
        for k in init_size:
            sizes[k] = init_size[k]
        for n in args:
            size = {}
            for name in args[n]:
                if name[0] not in sizes:
                    sizes[name[0]] = random.randint(2, 7)
                size[name] = sizes[name[0]]
            if "_s" in n:
                l = list(size.keys())[0]
                example[n] = pz.nx.arange(l, size[l])
            else:
                v = onp.random.randint(-5, 5, list(size.values()))
                example[n] = pz.nx.wrap(v).tag(*args[n])
        example = constraint(example)
        for n in args:
            x = example[n]
            x = x.untag(*args[n])
            reg[n] = x.unwrap().tolist()
            if len(args[n]) == 0:
                reg[n] = [0]
        return example, reg

    examples = []
    correct = 0
    for i in range(3):
        example, reg = make_instance()
        # out = example["return"].tolist()
        del example["return"]
        problem_spec(*reg.values())
        if len(reg["return"]) == 1:
            reg["return"] = reg["return"][0]
        yours = None
        yours = problem(**example)
        example["target"] = wrap(reg["return"])
        example["target"] = example["target"].tag(*args["return"])
        if yours is not None:
            example["yours"] = yours
        same = example["target"] == example["yours"]
        if same.untag(*same.named_shape.keys()).unwrap().all():
            correct += 1
        examples.append(example)
    if correct == 3:
        print("Correct")
    else:
        print("Failure")
    return examples


## Rules



1. Each puzzle needs to be solved in 1 line (<80 columns) of code.
2. You are only allowed to use `contract`, `where` and indexing.
3. You are *not allowed* anything else. No `view`, `sum`, `take`, `squeeze`, `tensor`.

### Example of named infix ops.

In [5]:
a = [arange("i", 4), arange("j", 5)]
b = [arange("j", 3), arange("i", 2)]

[{"a": a, "b":b, "ret": a + b} for a, b in zip(a, b)]

[{'a': <NamedArray int32(| i:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| i:4, j:3) [≥0, ≤5] zero:1 nonzero:11 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| j:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| j:5, i:2) [≥0, ≤5] zero:1 nonzero:9 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

### Example of where

In [6]:
examples = [(wrap([False, True], "i"), wrap([1, 1], "i"), wrap([-1, 0], "i")),
            (wrap([[False, True], [True, False]], "i", "j"), wrap([0, 1], "i"), wrap([-1, 0], "j")),
           ]
[{"q": q, "a":a, "b":b, "ret": where(q, a, b)} for q, a, b in examples]

[{'q': <NamedArray bool(| i:2) true:1 false:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'a': <NamedArray int32(| i:2) [≥1, ≤1] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:2) [≥-1, ≤0] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| i:2) [≥-1, ≤1] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'q': <NamedArray bool(| i:2, j:2) true:2 false:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'a': <NamedArray int32(| i:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:2) [≥-1, ≤0] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| i:2, j:2) [≥-1, ≤1] zero:2 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

### Example of contraction


In [7]:
def contract(n, *ts):
    t = 1
    for t2 in ts:
        t = t * t2
    return pz.nx.nmap(np.sum)(t.untag(n))

a = [arange("i", 4), arange("j", 5)]
b = [arange("j", 3), arange("i", 2)]

[{"a": a, "b":b, "ret": contract("i", a * b)} for a, b in zip(a, b)]

[{'a': <NamedArray int32(| i:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| j:3) [≥0, ≤12] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| j:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'ret': <NamedArray int32(| j:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 1 - ones

Compute [ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) - the vector of all ones.

In [8]:
def ones_spec(i_s, out):
    for i in i_s:
        out[i] = 1

def ones(i_s: Int32[NamedArray, "i"]) -> Int32[NamedArray, "i"]:
    return i_s * 0 + 1

make_test("one", ones, ones_spec)

Correct


[{'i_s': <NamedArray int32(| i:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:2) [≥1, ≤1] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:2) [≥1, ≤1] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i_s': <NamedArray int32(| i:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:6) [≥1, ≤1] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:6) [≥1, ≤1] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i_s': <NamedArray int32(| i:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:7) [≥1, ≤1] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:7) [≥1, ≤1] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 2 - sum

Compute [sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) - the sum of a vector.

In [9]:
def sum_spec(a, out):
    for i in range(len(a)):
        out[0] = out[0] + a[i]

def sum(a: Int32[NamedArray, "i"]) -> Int32[NamedArray, ""]:
    return contract("i", a)

make_test("sum", sum, sum_spec)

Correct


[{'a': <NamedArray int32(| i:6) [≥-5, ≤3] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray Array(-13, dtype=int32, weak_type=True) (wrapping jax.Array)>,
  'yours': <NamedArray Array(-13, dtype=int32) (wrapping jax.Array)>},
 {'a': <NamedArray int32(| i:7) [≥-3, ≤2] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray Array(-5, dtype=int32, weak_type=True) (wrapping jax.Array)>,
  'yours': <NamedArray Array(-5, dtype=int32) (wrapping jax.Array)>},
 {'a': <NamedArray int32(| i:4) [≥-4, ≤1] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray Array(-5, dtype=int32, weak_type=True) (wrapping jax.Array)>,
  'yours': <NamedArray Array(-5, dtype=int32) (wrapping jax.Array)>}]

## Puzzle 3 - outer

Compute [outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) - the outer product of two vectors.

In [10]:
def outer_spec(a, b, out):
    for i in range(len(out)):
        for j in range(len(out[0])):
            out[i][j] = a[i] * b[j]

def outer(a: Int32[NamedArray, "i"], b : Int32[NamedArray, "j"]) -> Int32[NamedArray, "i j"]:
    return a * b

make_test("outer", outer, outer_spec)

Correct


[{'a': <NamedArray int32(| i:4) [≥-2, ≤3] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:5) [≥-5, ≤4] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:4, j:5) [≥-15, ≤12] nonzero:20 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:4, j:5) [≥-15, ≤12] nonzero:20 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:4) [≥0, ≤4] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:2) [≥-5, ≤1] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:4, j:2) [≥-20, ≤4] zero:2 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:4, j:2) [≥-20, ≤4] zero:2 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:7) [≥-3, ≤4] zero:2 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| j:7) [≥-5, ≤4] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:7, j:7) [≥-20, ≤16] zero:19 nonzero:30 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:7, j:7) [≥-20, ≤16] zero:19 nonzero:30 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 4 - diag

Compute [diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html) - the diagonal vector of a square matrix.

In [11]:
def diag_spec(a, i1_s, out):
    for i in range(len(a)):
        out[i] = a[i][i]

def diag(a: Int32[NamedArray, "i1 i2"], i1_s: Int32[NamedArray, "i1"]) -> Int32[NamedArray, "i1"]:
    return a[{"i1": i1_s, "i2": i1_s}]


make_test("diag", diag, diag_spec)

Correct


[{'a': <NamedArray int32(| i1:2, i2:2) [≥-1, ≤2] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:2) [≥1, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:2) [≥1, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i1:5, i2:5) [≥-5, ≤4] zero:3 nonzero:22 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:5) [≥-4, ≤4] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:5) [≥-4, ≤4] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i1:3, i2:3) [≥-5, ≤3] nonzero:9 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:3) [≥-4, ≤2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:3) [≥-4, ≤2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 5 - eye

Compute [eye](https://numpy.org/doc/stable/reference/generated/numpy.eye.html) - the identity matrix.

In [25]:
def eye_spec(i1_s, i2_s, out):
    for i in i1_s:
        for j in i2_s:
            if i == j:
                out[i][j] = 1
            else:
                out[i][j] = 0

def eye(i1_s: Int32[NamedArray, "i1"], i2_s : Int32[NamedArray, "i2"]) -> Int32[NamedArray, "i1 i2"]:
    return i1_s == i2_s


make_test("eye", eye, eye_spec)

Correct


[{'i1_s': <NamedArray int32(| i1:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:7, i2:7) [≥0, ≤1] zero:42 nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:7, i2:7) true:7 false:42 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i1_s': <NamedArray int32(| i1:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:2, i2:2) [≥0, ≤1] zero:2 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:2, i2:2) true:2 false:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i1_s': <NamedArray int32(| i1:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:4, i2:4) [≥0, ≤1] zero:12 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:4, i2:4) true:4 false:12 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 6 - triu

Compute [triu](https://numpy.org/doc/stable/reference/generated/numpy.triu.html) - the upper triangular matrix.

In [26]:
def triu_spec(i1_s, i2_s, out):
    for i in i1_s:
        for j in i2_s:
            if i <= j:
                out[i][j] = 1
            else:
                out[i][j] = 0

def triu(i1_s: Int32[NamedArray, "i1"], i2_s : Int32[NamedArray, "i2"]) -> Int32[NamedArray, "i1 i2"]:
    return i1_s <= i2_s


make_test("triu", triu, triu_spec)

Correct


[{'i1_s': <NamedArray int32(| i1:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:4, i2:4) [≥0, ≤1] zero:6 nonzero:10 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:4, i2:4) true:10 false:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i1_s': <NamedArray int32(| i1:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:7, i2:7) [≥0, ≤1] zero:21 nonzero:28 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:7, i2:7) true:28 false:21 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'i1_s': <NamedArray int32(| i1:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i1:7, i2:7) [≥0, ≤1] zero:21 nonzero:28 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray bool(| i1:7, i2:7) true:28 false:21 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 7 - cumsum

Compute [cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) - the cumulative sum.

In [27]:
def cumsum_spec(a, i1_s, i2_s, out):
    total = 0
    for i in range(len(out)):
        out[i] = total + a[i]
        total += a[i]

def cumsum(a: Int32[NamedArray, "i1"], i1_s : Int32[NamedArray, "i1"], i2_s: Int32[NamedArray, "i2"]) -> Int32[NamedArray, "i2"]:
    return contract("i1", i1_s <= i2_s, a)

make_test("cumsum", cumsum, cumsum_spec)

Correct


[{'a': <NamedArray int32(| i1:3) [≥-5, ≤-2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:3) [≥-10, ≤-2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i2:3) [≥-10, ≤-2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i1:7) [≥-3, ≤3] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:7) [≥-4, ≤1] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i2:7) [≥-4, ≤1] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i1:7) [≥-5, ≤4] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:7) [≥-2, ≤5] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i2:7) [≥-2, ≤5] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 8 - diff

Compute [diff](https://numpy.org/doc/stable/reference/generated/numpy.diff.html) - the running difference.

In [28]:
def diff_spec(a, i_s, out):
    out[0] = a[0]
    for i in range(0, len(out)):
        out[i] = a[i] - a[(i - 1)]

def diff(a: Int32[NamedArray, "i"], i1_s : Int32[NamedArray, "i"]) -> Int32[NamedArray, "i"]:
    return a - a[{"i": i1_s - 1}]

make_test("diff", diff, diff_spec)

Correct


[{'a': <NamedArray int32(| i:3) [≥-5, ≤1] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:3) [≥-6, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:3) [≥-6, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:7) [≥-5, ≤4] nonzero:7 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i:7) [≥0, ≤6] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:7) [≥-6, ≤9] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:7) [≥-6, ≤9] zero:1 nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:5) [≥-4, ≤2] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:5) [≥-6, ≤3] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:5) [≥-6, ≤3] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 9 - stack

Compute [vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html) - the matrix of two vectors

In [30]:
def stack_spec(a, b, j_s, out):
    for i in range(len(out[0])):
        out[0][i] = a[i]
        out[1][i] = b[i]

def stack(a: Int32[NamedArray, "i"], b: Int32[NamedArray, "i"],
          j_s: Int32[NamedArray, "j"]) -> Int32[NamedArray, "j i"]:
    return where(j_s == 1, b, a)


make_test("stack", stack, stack_spec, init_size={"j" : 2})

Correct


[{'a': <NamedArray int32(| i:3) [≥-5, ≤1] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:3) [≥-4, ≤2] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:2, i:3) [≥-5, ≤2] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:2, i:3) [≥-5, ≤2] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:3) [≥-4, ≤3] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:3) [≥-3, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:2, i:3) [≥-4, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:2, i:3) [≥-4, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:2) [≥-3, ≤-2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'b': <NamedArray int32(| i:2) [≥2, ≤3] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:2, i:2) [≥-3, ≤3] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:2, i:2) [≥-3, ≤3] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 10 - roll

Compute [roll](https://numpy.org/doc/stable/reference/generated/numpy.roll.html) - the vector shifted 1 circular position.

In [17]:
def roll_spec(a, i_s, out):
    for i in range(len(out)):
        if i + 1 < len(out):
            out[i] = a[i + 1]
        else:
            out[i] = a[i + 1 - len(out)]

def roll(a: Int32[NamedArray, "i"], i_s: Int32[NamedArray, "i"]) -> Int32[NamedArray, "i"]:
    return a[{"i": (i_s + 1) % i_s.named_shape["i"]}]


make_test("roll", roll, roll_spec)

Correct


[{'a': <NamedArray int32(| i:6) [≥-4, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:6) [≥-4, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:6) [≥-4, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:5) [≥-4, ≤3] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:5) [≥-4, ≤3] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:5) [≥-4, ≤3] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:6) [≥-5, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:6) [≥-5, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:6) [≥-5, ≤4] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 11 - flip

Compute [flip](https://numpy.org/doc/stable/reference/generated/numpy.flip.html) - the reversed vector

In [32]:
def flip_spec(a, i_s, out):
    for i in range(len(out)):
        out[i] = a[len(out) - i - 1]

def flip(a: Int32[NamedArray, "i"], i_s: Int32[NamedArray, "i"]) -> Int32[NamedArray, "i"]:
    return a[{"i": -i_s - 1}]

make_test("flip", flip, flip_spec)

Correct


[{'a': <NamedArray int32(| i:3) [≥-5, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:3) [≥-5, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:3) [≥-5, ≤4] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:5) [≥-4, ≤0] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:5) [≥-4, ≤0] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:5) [≥-4, ≤0] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:2) [≥-2, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i:2) [≥-2, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i:2) [≥-2, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 12 - compress


Compute [compress](https://numpy.org/doc/stable/reference/generated/numpy.compress.html) - keep only masked entries (left-aligned).

In [19]:
def compress_spec(g, v, i1_s, i2_s, out):
    j = 0
    for i in range(len(out)):
        out[i] = 0
    for i in range(len(g)):
        if g[i] > 1:
            out[j] = v[i]
            j += 1

def compress(g: Int32[NamedArray, "i1"], v: Int32[NamedArray, "i2"], i1_s:Int32[NamedArray, "i1"], i2_s:Int32[NamedArray, "i2"]) -> Int32[NamedArray, "i2"]:
    # I don't know how to do this one!
    return g

make_test("compress", compress, compress_spec)

Failure


[{'g': <NamedArray int32(| i1:5) [≥-5, ≤4] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'v': <NamedArray int32(| i2:5) [≥-4, ≤2] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:5) [≥-3, ≤0] zero:4 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:5) [≥-5, ≤4] nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'g': <NamedArray int32(| i1:5) [≥-5, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'v': <NamedArray int32(| i2:5) [≥-4, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:5) [≥0, ≤4] zero:4 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:5) [≥-5, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'g': <NamedArray int32(| i1:6) [≥-5, ≤3] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'v': <NamedArray int32(| i2:6) [≥-5, ≤2] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i1_s': <NamedArray int32(| i1:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i2_s': <NamedArray int32(| i2:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| i2:6) [≥-4, ≤0] zero:5 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| i1:6) [≥-5, ≤3] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 13 - pad_to


Compute pad_to - eliminate or add 0s to change size of vector.

In [33]:
def pad_to_spec(a, i_s, j_s, out):
    for i in range(len(out)):
        if i < len(a):
            out[i] = a[i]
        else:
            out[i] = 0

def pad_to(a: Int32[NamedArray, "i"], i_s:Int32[NamedArray, "i"], j_s:Int32[NamedArray, "j"])  -> Int32[NamedArray, "j"]:
    return contract("i", a, j_s == i_s)


make_test("pad_to", pad_to, pad_to_spec)

Correct


[{'a': <NamedArray int32(| i:3) [≥-5, ≤0] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:3) [≥-5, ≤0] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:3) [≥-5, ≤0] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:6) [≥-5, ≤3] nonzero:6 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:3) [≥-5, ≤3] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:3) [≥-5, ≤3] nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:4) [≥-4, ≤2] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j_s': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 14 - sequence_mask


Compute [sequence_mask](https://www.tensorflow.org/api_docs/python/tf/sequence_mask) - pad out to length per batch.

In [21]:
# Didn't do
# def sequence_mask_spec(values, length, out):
#     for i in range(len(out)):
#         for j in range(len(out[0])):
#             if j < length[i]:
#                 out[i][j] = values[i][j]
#             else:
#                 out[i][j] = 0

# def sequence_mask(values: TT["i", "j"], length: TT["i", int]) -> TT["i", "j"]:
#     pass


# def constraint_set_length(d):
#     d["length"] = d["length"] % d["values"].shape[1]
#     return d

# make_test("sequence_mask",
#     sequence_mask, sequence_mask_spec, constraint=constraint_set_length
# )

## Puzzle 15 - bincount

Compute [bincount](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html) - count number of times an entry was seen.

In [35]:
def bincount_spec(a, i_s, j1_s, j2_s, out):
    for i in range(len(out)):
        out[i] = 0
    for i in range(len(a)):
        out[a[i]] += 1

def bincount(a: Int32[NamedArray, "i"], i_s: Int32[NamedArray, "i"],
             j1_s: Int32[NamedArray, "j1"], j2_s: Int32[NamedArray, "j2"]) -> Int32[NamedArray, "j2"]:
    return contract("i", (j1_s == j2_s)[{"j1": a}])


def constraint_set_max(d):
    d["a"] = d["a"] % d["return"].named_shape["j2"]
    return d


make_test("bincount",
    bincount, bincount_spec, constraint=constraint_set_max
)

Correct


[{'a': <NamedArray int32(| i:5) [≥0, ≤3] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:4) [≥1, ≤2] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:4) [≥1, ≤2] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:4) [≥0, ≤1] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:2) [≥0, ≤1] zero:1 nonzero:1 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:2) [≥1, ≤3] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:2) [≥1, ≤3] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'a': <NamedArray int32(| i:3) [≥0, ≤3] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'i_s': <NamedArray int32(| i:3) [≥0, ≤2] zero:1 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:4) [≥0, ≤3] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:4) [≥0, ≤1] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:4) [≥0, ≤1] zero:1 nonzero:3 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

## Puzzle 16 - scatter_add

Compute [scatter_add](https://pytorch-scatter.readthedocs.io/en/1.3.0/functions/add.html) - add together values that link to the same location.

In [36]:
def scatter_add_spec(values, link, j1_s, j2_s, out):
    for i in range(len(out)):
        out[i] = 0
    for j in range(len(values)):
        out[link[j]] += values[j]

def scatter_add(values: Int32[NamedArray, "i"], link: Int32[NamedArray,"i"],
                j1_s: Int32[NamedArray, "j1"], j2_s: Int32[NamedArray, "j2"]) -> Int32[NamedArray, "j2"]:
    return contract("i", values, (j1_s == j2_s)[{"j1": link}])


def constraint_set_max(d):
    d["link"] = d["link"] % d["return"].named_shape["j2"]
    return d

make_test("scatter_add",
    scatter_add, scatter_add_spec, constraint=constraint_set_max
)

Correct


[{'values': <NamedArray int32(| i:4) [≥-3, ≤0] zero:2 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'link': <NamedArray int32(| i:4) [≥1, ≤4] nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:5) [≥0, ≤4] zero:1 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:5) [≥-3, ≤0] zero:3 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:5) [≥-3, ≤0] zero:3 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'values': <NamedArray int32(| i:2) [≥1, ≤2] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'link': <NamedArray int32(| i:2) [≥1, ≤4] nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:6) [≥0, ≤2] zero:4 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:6) [≥0, ≤2] zero:4 nonzero:2 (wrapping jax.Array)
    <Arrayviz rendering>
  >},
 {'values': <NamedArray int32(| i:6) [≥-5, ≤2] zero:2 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'link': <NamedArray int32(| i:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j1_s': <NamedArray int32(| j1:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'j2_s': <NamedArray int32(| j2:6) [≥0, ≤5] zero:1 nonzero:5 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'target': <NamedArray int32(| j2:6) [≥-5, ≤2] zero:2 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >,
  'yours': <NamedArray int32(| j2:6) [≥-5, ≤2] zero:2 nonzero:4 (wrapping jax.Array)
    <Arrayviz rendering>
  >}]

In [24]:
import inspect
fns = (ones, sum, outer, diag, eye, triu, cumsum, diff, stack, roll, flip,
       compress, pad_to,  bincount, scatter_add)

for fn in fns:
    lines = [l for l in inspect.getsource(fn).split("\n") if not l.strip().startswith("#")]

    if len(lines) > 3:
        print(fn.__name__, len(lines[2]), "(more than 1 line)")
    else:
        print(fn.__name__, len(lines[1]))

ones 22
sum 27
outer 16
diag 38
eye 36
triu 36
cumsum 55
diff 33
stack 43
roll 53
flip 31
compress 12
pad_to 52
bincount 52 (more than 1 line)
scatter_add 63 (more than 1 line)
